In [1]:
import json
import pandas as pd
import os

In [4]:
def update_provincia_ids(json_path, csv_path, output_path):
    """
    Carga el GeoJSON de provincias y el CSV de códigos de provincia,
    y actualiza cada feature['properties']['id'] e 'id' con el código INE.
    """
    # 1) Leer GeoJSON
    with open(json_path, 'r', encoding='utf-8') as f:
        geo = json.load(f)

    # 2) Leer CSV de códigos
    df_codes = pd.read_csv(csv_path, dtype=str, delimiter=";")
    mapping = dict(zip(df_codes['name'], df_codes['cod_prov']))

    # 3) Actualizar cada feature
    missing = []
    for feat in geo.get('features', []):
        prov_name = feat['properties'].get('name')
        cod = mapping.get(prov_name)
        if cod:
            feat['properties']['id'] = cod
            feat['id'] = cod
        else:
            missing.append(prov_name)

    if missing:
        print(f"Advertencia: No se encontró código para: {sorted(set(missing))}")

    # 4) Guardar nuevo GeoJSON
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(geo, f, ensure_ascii=False, indent=2)

In [5]:
if __name__ == "__main__":
    json_input  = "../data/mapas/esp_prov.json"
    csv_input   = "../data/codigos_ine_prov_ccaa/codigos_prov_mapa.csv"
    json_output = "..data/mapas/esp_prov_codigos.json"
    update_provincia_ids(json_input, csv_input, json_output)
    print(f"GeoJSON actualizado guardado en: {json_output}")

GeoJSON actualizado guardado en: ..data/mapas/esp_prov_codigos.json
